In [1]:
import os
import pandas as pd
import pickle

import hdbscan
import pandas as pd

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
from bertopic.vectorizers import ClassTfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


c:\Users\DNA_A\hsx\healthhub-content-optimization\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DNA_A\hsx\healthhub-content-optimization\.venv\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\DNA_A\hsx\healthhub-content-optimization\.venv\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [37]:
with open(os.path.join(experiment_path,"neo4j_predicted_clusters.pkl"), "rb") as f:
    emb_results = pickle.load(f)
pd.DataFrame(emb_results)

,cluster,titles,cluster_keywords
0,13,"[Slip-Ups: Dos and Dont's, Use This Coupon: Get Support from Your Loved Ones]","[milestone, worth, 28th, supporter, evouchers]"
1,26,"[Cooking Healthy For Deepavali, 6 Tips To Healthy Deepavali Eating Whether As A Host Or Guest, Choosing Healthy Hari Raya Food and Deepavali Snacks, Eating Healthy During Deepavali]","[deepavali, indian, festive, bake, festival]"
2,34,"[Blogger Kenneth Lee’s Amazing Weight-loss Journey, Losing Weight by Eating Well - One Man's Successful Weight Loss Story, Healthy Food Choices Is Helping Him Lose The Kilos]","[mr, teo, ltw, kenneth, wu]"
3,36,"[Apple Seafood Fried Brown Rice, Wok Fried Broccoli with Assorted Mushroom]","[tbsp, fry, 1cm, stir, seasoning]"
4,52,"[Preventing HPV infection: HPV Vaccination, Cervical Cancer]","[hpv, cervical, cancer, vaccination, vaccine]"
5,54,"[Herpes: Causes, Symptoms, and Treatment, Genital Herpes: Symptoms, Causes and Treatments]","[herpes, hsv, genital, sore, blister]"
6,56,"[Hepatitis B, Hepatitis A]","[hepatitis, liver, hav, infected, virus]"
7,59,"[Pneumonia, Pneumococcal Disease]","[pneumococcal, pneumonia, lung, 65, vaccination]"
8,74,"[Influenza, Mumps: Causes, Symptoms, and Treatment, Avian Influenza H5N1 (Bird Flu), Avian Influenza H7N9 (Bird Flu), All You Need to Know About Childhood Immunisations, Immunisation: inject to protect, Does Your Child Need The Influenza Vaccine?, Start Your Year-End Vacation With a Flu Jab, 4 Reasons to Get Vaccinated—Adult Immunisation, Know How to Fight Flu, Mumps Vaccine: Why We Want to Prevent Mumps, Protect Yourself and Others Against the Flu, Beat the Flu When You Have Diabetes]","[flu, influenza, vaccine, mumps, vaccination]"
9,117,"[Fever in Children, Hand, Foot, and Mouth Disease, Childhood Illnesses: 10 Most Common Conditions in Children, Baby Colic, Stomach Flu in Children, Cough and the Common Cold In Children, Constipation in Children, Asthma (Common Childhood Illnesses), Food Poisoning in Children, Common Childhood Injuries and Childhood Injury Prevention, Common childhood conditions – Gastroenteritis, Guide to Infant Medication, Feeding your baby: Breastfeeding, Q&A: Is My Baby Constipated?, Q&A: My Baby’s Stools Are Watery, What Should I Do?, Common infant problems and conditions, Surviving Sleep Deprivation With a Baby, The Beginner’s Guide To Taking Care of Your Baby: 0 to 6 Months, Guide to Common Infant Conditions, Your Growing Baby, The Importance of Sleep, Head Injuries in Children, 7 Reasons Why Your Baby's Crying, Care of Your Newborn: To Have and to Hold, Guide to Toddler Medication, Gearing up for birth, Q&A: Help! My Baby Is Coughing and Wheezing!, Why Is Sleep Important for Kids?, What To Eat While Breastfeeding, Outdoor Activities for Your Children, The Beginner’s Guide To Taking Care of Your Toddler: 19 to 24 Months, First days of our lives with baby, Activities for 6-12 Months: Let’s Play, Mummy!, 3 Ways to Improve Sleep Quality, Q&A: Is My Child Having a Cold?, When Does My Child Need Antibiotics?, Getting Fit Through Fun for kids, Guide to Common Toddler Conditions, Toddlers and Physical Activity – What You Need To Know, The Beginner’s Guide To Taking Care of Your Infant: 7 to 12 Months, Baby Up and About Safely, Coping with your two-year-old: when a firm hand is needed, Your baby needs soft skills too, Antimicrobial Resistance: All You Need to Know, Help Your Baby Develop, The Beginner’s Guide To Taking Care of Your Toddler: 13 to 18 Months, Let’s Go, Mummy & Daddy!, Activity Ideas to Get Your Tot Moving, Prevent Falls and Keep Your Child Injury-Free, Baby's here! What's next?, Safe Home for Your Baby, Baby has come home: dealing with changes, Q&A: How Can I Get My Baby to Sleep Well and Safely?, Q&A: Help! My Baby Won’t Stop Crying!, Keeping your child safe, Fun, Family Activities for 6-Year-Olds, Activity Ideas to Get Your 5-Year-Old Moving, Activity Ideas to Get Your 3-Year-Old Moving, Activity Ideas to Get Your Pre-Schooler Moving, Growing Up Safe,

In [2]:
root_path = "../data"
experiment_path = os.path.join(root_path,"07_model_output","all-MiniLM-L6-v2","experiment-5b-weighted-similarities-3_title_7_body")
weighted_embeddings_path = os.path.join(root_path, "04_feature", "weighted_embeddings.pkl")

with open(weighted_embeddings_path, "rb") as f:
    weighted_embeddings = pickle.load(f)

pred_cluster_df = pd.read_csv(os.path.join(experiment_path, "predicted_cluster.csv"))

In [3]:
cluster_size_count = pred_cluster_df.cluster.value_counts()
to_keep = cluster_size_count[cluster_size_count >10].index
cluster_morethan10 = pred_cluster_df[pred_cluster_df.cluster.isin(to_keep)]
print('No. of cluster to subgroup: ', cluster_morethan10.cluster.nunique())

No. of cluster to subgroup:  10


In [4]:
cluster_morethan10_embeddings = pd.merge(
    cluster_morethan10,
    weighted_embeddings[['id','vector_extracted_content_body']],
    how='left',
    on='id')

print(cluster_morethan10.shape[0] == cluster_morethan10.shape[0])

True


In [5]:
cluster_morethan10.cluster.value_counts()

cluster
117    63
461    53
171    50
606    39
196    35
629    27
482    21
74     13
275    13
195    12
Name: count, dtype: int64

In [6]:
def get_embeddings(cluster_df):
    embeddings = np.array(cluster_df.vector_extracted_content_body.to_list())
    doc_titles = cluster_df.title.to_list()
    docs = cluster_df.body_content.to_list()
    ids = cluster_df.id.to_list()
    umap_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine', random_state=42)
    umap_embeddings = umap_model.fit_transform(embeddings)

    return embeddings, doc_titles, docs, ids, umap_embeddings

In [7]:
def hyperparameter_tuning(embeddings):
    best_score = 0

    for min_cluster_size in [2,3,4,5,6]:
        for min_samples in [1,2,3,4,5,6,7]:
            for cluster_selection_method in ['leaf']:
                for metric in ['euclidean','manhattan']:
                    # for each combination of parameters of hdbscan
                    hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,min_samples=min_samples,
                                        cluster_selection_method=cluster_selection_method, metric=metric, 
                                        gen_min_span_tree=True).fit(embeddings)
                    # DBCV score
                    score = hdb.relative_validity_
                    if score > best_score:
                        best_score = score
                        best_parameters = {'min_cluster_size': min_cluster_size, 
                                'min_samples':  min_samples, 'cluster_selection_method': cluster_selection_method,
                                'metric': metric}

    print("Best DBCV score: {:.3f}".format(best_score))
    print("Best parameters: {}".format(best_parameters))
    return best_parameters

In [8]:
def topic_modelling(hyperparameters):
    # Step 3 - Cluster reduced embeddings
    hdbscan_model = HDBSCAN(min_cluster_size=hyperparameters['min_cluster_size'], min_samples=hyperparameters['min_samples'], metric=hyperparameters['metric'], cluster_selection_method=hyperparameters['cluster_selection_method'], prediction_data=True, gen_min_span_tree=True)

    # Step 4 - Tokenize topics
    vectorizer_model = CountVectorizer(stop_words="english")

    # Step 5 - Create topic representation
    ctfidf_model = ClassTfidfTransformer()

    # Step 6 - (Optional) Fine-tune topic representations with 
    representation_model = MaximalMarginalRelevance(diversity=0.3)

    # All steps together
    topic_model = BERTopic(
    # embedding_model=embedding_model,          # Step 1 - Extract embeddings
    # umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic represenations
    # nr_topics="auto" #default is none, will auto reduce topics using HDBSCAN
    )
    return topic_model

In [9]:
cluster_morethan10_embeddings.cluster.value_counts()

cluster
117    63
461    53
171    50
606    39
196    35
629    27
482    21
74     13
275    13
195    12
Name: count, dtype: int64

In [30]:
def create_topic_assigner(start_counter):
    counter = start_counter
    
    def assign_new_topic(x):
        nonlocal counter
        if x == -1:
            new_topic = counter
            counter += 1
            return new_topic
        else:
            return x

    return assign_new_topic


def process_cluster(cluster_df):
    embeddings, doc_titles, docs, ids, umap_embeddings = get_embeddings(cluster_df)
    hyperparameters = hyperparameter_tuning(umap_embeddings)
    topic_model = topic_modelling(hyperparameters)
    topics, _ = topic_model.fit_transform(docs, embeddings)

    top_n = 50
    top_topics = topic_model.get_topic_freq().head(top_n)['Topic'].tolist()
    reduced_embeddings = topic_model.umap_model.embedding_
    hover_data = [f"{title} - Topic {topic}" for title, topic in zip(doc_titles, topics)]
    # visualization = topic_model.visualize_documents(hover_data, reduced_embeddings=reduced_embeddings, topics=top_topics, title=f'Top {top_n} Topics') 
    # visualization.show() 

    # visualization_barchart = topic_model.visualize_barchart(top_n_topics=top_n)
    # visualization_barchart.show()

    result_df = pd.DataFrame({"Assigned Topic": topics, "Title": doc_titles, "id": ids})
    topic_kws = topic_model.get_topic_info()[['Topic', 'Representation']]
    topic_kws['top_5_kws'] = topic_kws['Representation'].apply(lambda x: x[:5])
    result_df_kws = pd.merge(result_df, topic_kws, how='left', left_on='Assigned Topic', right_on='Topic')
    result_df_kws = result_df_kws.drop(['Representation', 'Topic'], axis=1)
    result_df_kws = result_df_kws[['Id', 'Title', 'Assigned Topic', 'top_5_kws']]

    max_topic = result_df_kws['Assigned Topic'].max()
    new_topic_counter = max_topic + 1

    assign_new_topic_func = create_topic_assigner(new_topic_counter)
    result_df_kws['Assigned Topic'] = result_df_kws['Assigned Topic'].apply(assign_new_topic_func)

    cluster_id = cluster_df['cluster'].unique()[0]
    result_df_kws['Assigned Topic'] = result_df_kws['Assigned Topic'].apply(lambda x: 'Cluster_' + str(cluster_id) + '_' + str(x))

    return result_df_kws

def process_all_clusters(cluster_morethan10_embeddings):
    unique_clusters = cluster_morethan10_embeddings['cluster'].unique()
    all_results = []

    for cluster_id in unique_clusters:
        cluster_df = cluster_morethan10_embeddings[cluster_morethan10_embeddings['cluster'] == cluster_id]
        result_df_kws = process_cluster(cluster_df)
        all_results.append(result_df_kws)

    combined_df = pd.concat(all_results, ignore_index=True)
    return combined_df

def assign_unique_numbers_to_topics(final_result_df, pred_cluster_df):
    max_cluster_value = pred_cluster_df['cluster'].max()
    unique_assigned_topics = final_result_df['Assigned Topic'].unique()
    topic_number_mapping = {topic: idx + max_cluster_value + 1 for idx, topic in enumerate(unique_assigned_topics)}
    
    final_result_df['Assigned Topic Number'] = final_result_df['Assigned Topic'].map(topic_number_mapping)
    return final_result_df

final_result_df = process_all_clusters(cluster_morethan10_embeddings)
final_result_df_with_numbers = assign_unique_numbers_to_topics(final_result_df, pred_cluster_df)

Best DBCV score: 0.258
Best parameters: {'min_cluster_size': 2, 'min_samples': 1, 'cluster_selection_method': 'leaf', 'metric': 'manhattan'}
Best DBCV score: 0.420
Best parameters: {'min_cluster_size': 3, 'min_samples': 6, 'cluster_selection_method': 'leaf', 'metric': 'euclidean'}
Best DBCV score: 0.403
Best parameters: {'min_cluster_size': 2, 'min_samples': 4, 'cluster_selection_method': 'leaf', 'metric': 'manhattan'}
Best DBCV score: 0.254
Best parameters: {'min_cluster_size': 2, 'min_samples': 1, 'cluster_selection_method': 'leaf', 'metric': 'euclidean'}
Best DBCV score: 0.324
Best parameters: {'min_cluster_size': 5, 'min_samples': 1, 'cluster_selection_method': 'leaf', 'metric': 'euclidean'}
Best DBCV score: 0.318
Best parameters: {'min_cluster_size': 3, 'min_samples': 1, 'cluster_selection_method': 'leaf', 'metric': 'manhattan'}
Best DBCV score: 0.910
Best parameters: {'min_cluster_size': 5, 'min_samples': 6, 'cluster_selection_method': 'leaf', 'metric': 'manhattan'}
Best DBCV sco

In [56]:
new_cluster_to_merge = final_result_df_with_numbers[['Id','top_5_kws','Assigned Topic Number']]
new_cluster_to_merge.columns = ['Id','cluster_kws','new_cluster']
updated_pred_cluster = pd.merge(pred_cluster_df, new_cluster_to_merge, how='left', on='id')

id  \
0    1445475   
1    1445725   
2    1445247   
3    1445260   
4    1444610   
..       ...   
663  1444872   
664  1444991   
665  1443375   
666  1445000   
667  1443220   

                                                                   title  \
0                            Emotional Support for Those Living with HIV   
1                                       The Facts of Childhood Fractures   
2                                     30 Ways to Go from Uncle to Hunkle   
3                             Age is only a Number - Move it, Feel young   
4                                               Slip-Ups: Dos and Dont's   
..                                                                   ...   
663                                    Q&A: Is My Baby's Spit-Up Normal?   
664         Baby’s Second Year: Bite-sized Reminders for Mummy and Daddy   
665                                            Stay Active As You Travel   
666        Child’s Fourth Year: Bite-sized Reminders for Mummy and Daddy   
667  How to Get Your Parents to Get Moving and Adopt an Active Lifestyle   

                                                                               url  \
0                       https://www.healthhub.sg/live-healthy/being_a_helping_hand   
1                            https://www.healthhub.sg/live-healthy/Child-fractures   
2    https://www.healthhub.sg/live-healthy/mens-health-uncle-to-hunkle-bro-bonding   
3                         https://www.healthhub.sg/live-healthy/move-it-feel-young   
4                     https://www.healthhub.sg/live-healthy/slip-ups-dos-and-donts   
..                                                                             ...   
663              https://www.healthhub.sg/live-healthy/is-my-baby's-spit-up-normal   
664    https://www.healthhub.sg/live-healthy/baby-second-year-bite-sized-reminders   
665                https://www.healthhub.sg/live-healthy/stay-active-as-you-travel   
666   https://www.healthhub.sg/live-healthy/child-fourth-year-bite-sized-reminders   
667    https://www.healthhub.sg/live-healthy/how-to-get-your-parents-to-get-moving   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
pred_cluster_df['cluster'].apply(lambda x: )

In [31]:
pd.set_option('display.max_rows', 400)
final_result_df_with_numbers.sort_values('Assigned Topic Number')

,Id,Title,Assigned Topic,top_5_kws,Assigned Topic Number
0,1437295,Influenza,Cluster_74_0,"[flu, influenza, vaccination, children, vaccine]",665
10,1443007,4 Reasons to Get Vaccinated—Adult Immunisation,Cluster_74_0,"[flu, influenza, vaccination, children, vaccine]",665
9,1444584,Start Your Year-End Vacation With a Flu Jab,Cluster_74_0,"[flu, influenza, vaccination, children, vaccine]",665
8,1445298,Does Your Child Need The Influenza Vaccine?,Cluster_74_0,"[flu, influenza, vaccination, children, vaccine]",665
11,1445720,Mumps Vaccine: Why We Want to Prevent Mumps,Cluster_74_2,"[mumps, pain, glands, parotid, swelling]",666
1,1437304,"Mumps: Causes, Symptoms, and Treatment",Cluster_74_2,"[mumps, pain, glands, parotid, swelling]",666
3,1437505,Avian Influenza H7N9 (Bird Flu),Cluster_74_3,"[avian, influenza, birds, bird, poultry]",667
2,1437510,Avian Influenza H5N1 (Bird Flu),Cluster_74_3,"[avian, influenza, birds, bird, poultry]",667
5,1442805,Immunisation: inject to protect,Cluster_74_4,"[vaccines, injection, baby, immunisation, vaccinations]",668
4,1443987,All You Need to Know About Childhood Immunisations,Cluster_74_4,"[vaccines, injection, baby, immunisation, vaccinations]",668
